In [35]:
import pandas as pd
df = pd.read_csv("./data/e2e_table.csv")
display(df)

,dataset,type,acc,mean_s,sum_s,bandwidth_GB,mean_factor,bw_factor
0,CIFAR Large,l2_opt,0.816200,149.268409,23882.945409,70.71,6.86,4
1,CIFAR Large,linf_opt,0.849900,63.506440,10161.030440,143.15,2.92,8
2,CIFAR Large,plain,0.857900,21.748937,3479.829937,16.89,1.00,1
3,CIFAR Small,l2,0.602700,110.473556,11047.355556,68.28,59.71,28
4,CIFAR Small,l2_opt,0.586100,18.171788,1817.178788,13.51,9.82,6
5,CIFAR Small,linf,0.603000,86.193707,8619.370707,20.66,46.58,8
6,CIFAR Small,linf_opt,0.607200,14.550101,1455.010101,20.58,7.86,8
7,CIFAR Small,plain,0.607300,1.850263,185.026263,2.45,1.00,1
8,MNIST,l2,0.986500,47.194950,7551.191950,34.20,20.96,26
9,MNIST,l2_opt,0.973400,10.682082,1709.133082,9.33,4.74,7


In [36]:
import numpy as np

rounds = {
            'MNIST': 160,
            'CIFAR Small': 100,
             'CIFAR Large': 160,
             'Shakespeare': 20
         }

to_extrapolate = {
    'CIFAR Large': {
        'linf_ext': {
            "timings": [792, 804],
            "bandwidth": 143.37,
            "accuracy": df[(df['dataset'] == "CIFAR Large") & (df['type'] == 'linf_opt')]["acc"].item()
        },
        'l2_ext': {
            "timings": [703, 698, 687, 695, 694], # [2277.959, 2278.978, 2253.280, 2268.361, 2262.207, 2158.015],
            "bandwidth": 478.92,
            "accuracy": 0.85
        }
    },
    'Shakespeare': {
        'linf_ext': {
            "timings": [2397], #average from 5 rounds
            "bandwidth": 53.49,
            "accuracy": df[(df['dataset'] == "Shakespeare") & (df['type'] == 'linf_opt')]["acc"].item()
        },
        'l2_ext': {
            "timings": [2526], #average from 5 rounds
            "bandwidth": 179.12,
            "accuracy": df[(df['dataset'] == "Shakespeare") & (df['type'] == 'linf_opt')]["acc"].item()
        }
    }
}

for ext, rows in to_extrapolate.items():
    for bound_type, data in rows.items():
        factor_row = df[(df['dataset'] == ext) & (df['type'] == 'plain')]

        mean_s = np.mean(data["timings"])
        sum_s = mean_s * float(rounds[ext])
        bw = data['bandwidth']

        mean_factor = int(round(mean_s / factor_row['mean_s'].item(), 2))
        bw_factor = int(round(bw / factor_row['bandwidth_GB'].item(), 0))

        df_extrapolated = pd.DataFrame(
            [
                [ext, bound_type, data["accuracy"], mean_s, sum_s,
                 bw, mean_factor, bw_factor]
            ], columns=['dataset', 'type', 'acc', 'mean_s', 'sum_s',
                                          'bandwidth_GB', 'mean_factor', 'bw_factor']
        )
        df = df.append(df_extrapolated)

display(df)

,dataset,type,acc,mean_s,sum_s,bandwidth_GB,mean_factor,bw_factor
0,CIFAR Large,l2_opt,0.816200,149.268409,23882.945409,70.71,6.86,4
1,CIFAR Large,linf_opt,0.849900,63.506440,10161.030440,143.15,2.92,8
2,CIFAR Large,plain,0.857900,21.748937,3479.829937,16.89,1.00,1
3,CIFAR Small,l2,0.602700,110.473556,11047.355556,68.28,59.71,28
4,CIFAR Small,l2_opt,0.586100,18.171788,1817.178788,13.51,9.82,6
5,CIFAR Small,linf,0.603000,86.193707,8619.370707,20.66,46.58,8
6,CIFAR Small,linf_opt,0.607200,14.550101,1455.010101,20.58,7.86,8
7,CIFAR Small,plain,0.607300,1.850263,185.026263,2.45,1.00,1
8,MNIST,l2,0.986500,47.194950,7551.191950,34.20,20.96,26
9,MNIST,l2_opt,0.973400,10.682082,1709.133082,9.33,4.74,7


In [37]:
# extrapolate cifar l2 opt
# multiplier = float(158) / float(26)
# df[(df['dataset'] == 'CIFAR Large') & (df['type'] == 'l2_opt')]['sum_s'] = df[(df['dataset'] == 'CIFAR Large') & (df['type'] == 'l2_opt')]['sum_s'] * multiplier
# df[(df['dataset'] == 'CIFAR Large') & (df['type'] == 'l2_opt')]['acc'] = 0.82

# df[df['dataset'] == 'CIFAR Large' & df['type'] == 'l2_opt']['mean_s'] = df[df['dataset'] == 'CIFAR Large' & df['type'] == 'l2_opt']['sum_s'] * multiplier

In [38]:
header = r"""
\begin{table}
    \centering
    \resizebox{\columnwidth}{!}{%
    \begin{tabular}{lrcrrrcrr}
        \toprule
             &                && \multicolumn{3}{c}{Computation Time} & & \multicolumn{2}{c}{Bandwidth} \\
             \cline{4-6}\cline{8-9}
        Type & Acc. && Round [s] & Total [m] & Factor & & Total [MB] & Factor \\"""


mnist_header = r"""        \toprule
        \multicolumn{9}{c}{\textsc{MNIST} (19k params, rsl 5k params, 160 rounds)} \\"""

cifar_small_header = r"""        \toprule
        \multicolumn{9}{c}{\textsc{CIFAR-10 S} (62k params, rsl 12k params, 100 rounds)} \\"""

cifar_large_header = r"""        \toprule
        \multicolumn{9}{c}{\textsc{CIFAR-10 L} (273k params, rsl 40k params, 160 rounds)} \\"""

shakespeare_large_header = r"""        \toprule
        \multicolumn{9}{c}{\textsc{Shakespeare} (818k params, 20 rounds)} \\"""

footer = r"""        \bottomrule
    \end{tabular}
    }
\end{table}
"""

type_order = {
    "plain": 0,
    "l2": 1,
    "l2_ext": 2,
    "l2_opt": 3,
    "l2_opt_ext": 4,
    "linf": 5,
    "linf_ext": 6,
    "linf_opt": 7,
    "linf_opt_ext": 8
}

df['type_order'] = df['type'].map(type_order)
df = df.sort_values(by=["dataset", "type_order"])

type_print = {
    "plain": r"SA",
    "l2": r"$L_2$",
    "l2_opt": r"$L_2^{(rsl)}$",
    "linf": r"$L_\infty$",
    "linf_opt": r"$L_\infty^{(p)}$",

    "linf_ext": r"$L_\infty$",
    "l2_ext": r"$L_2$"
}

type_extrapolated = [
    "linf_ext", "l2_ext"
]

def format_mean_factor(factor):
    if factor > 1.01 and factor < 3.0:
        return f"{round(factor, 1):.1f}"
    return f"{round(factor, 0):.0f}"

def format_row(x):
    num_rounds = rounds[x.dataset]
    num_clients = 48
    def format_per_client_per_round(v):
        return round(float(v) * 1000.0 / (num_rounds * num_clients), 1)
    
    if x.type in type_extrapolated:
        return fr"{type_print[x.type]} * & {round(x.acc, 2):.2f} & & {round(x.mean_s)} & {round(x.sum_s/60)} & {format_mean_factor(x.mean_factor)}x & & {format_per_client_per_round(x.bandwidth_GB)} & {x.bw_factor}x \\"

    # total_time = f"{round(x.sum_s/60)}m" if round(x.sum_s/60) > 0 else f"{round(x.sum_s)}s"
    return fr"{type_print[x.type]} & {round(x.acc, 2):.2f} & & {round(x.mean_s)} & {round(x.sum_s/60)} & {format_mean_factor(x.mean_factor)}x & & {format_per_client_per_round(x.bandwidth_GB)} & {x.bw_factor}x \\"

# PRINT THE LATEX TABLE

print(header)

print(mnist_header)


for row in df[df["dataset"]=="MNIST"].itertuples():
    print("        " + format_row(row))

print(cifar_small_header)
    
for row in df[df["dataset"]=="CIFAR Small"].itertuples():
    print("        " + format_row(row))

print(cifar_large_header)
    
for row in df[df["dataset"]=="CIFAR Large"].itertuples():
    print("        " + format_row(row))

print(shakespeare_large_header)

for row in df[df["dataset"]=="Shakespeare"].itertuples():
    print("        " + format_row(row))
    
print(footer)




\begin{table}
    \centering
    \resizebox{\columnwidth}{!}{%
    \begin{tabular}{lrcrrrcrr}
        \toprule
             &                && \multicolumn{3}{c}{Computation Time} & & \multicolumn{2}{c}{Bandwidth} \\
             \cline{4-6}\cline{8-9}
        Type & Acc. && Round [s] & Total [m] & Factor & & Total [MB] & Factor \\
        \toprule
        \multicolumn{9}{c}{\textsc{MNIST} (19k params, rsl 5k params, 160 rounds)} \\
        SA & 0.99 & & 2 & 6 & 1x & & 0.2 & 1x \\
        $L_2$ & 0.99 & & 47 & 126 & 21x & & 4.5 & 26x \\
        $L_2^{(rsl)}$ & 0.97 & & 11 & 28 & 5x & & 1.2 & 7x \\
        $L_\infty$ & 0.99 & & 40 & 108 & 18x & & 1.4 & 8x \\
        $L_\infty^{(p)}$ & 0.99 & & 8 & 21 & 3x & & 1.4 & 8x \\
        \toprule
        \multicolumn{9}{c}{\textsc{CIFAR-10 S} (62k params, rsl 12k params, 100 rounds)} \\
        SA & 0.61 & & 2 & 3 & 1x & & 0.5 & 1x \\
        $L_2$ & 0.60 & & 110 & 184 & 60x & & 14.2 & 28x \\
        $L_2^{(rsl)}$ & 0.59 & & 18 & 30 & 10x & & 